# Importing of library 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
lm=XGBClassifier()

# Loding datasets

In [2]:
coupon=pd.read_csv('coupon_item_mapping.csv')
item_data=pd.read_csv('item_data.csv')
customer=pd.read_csv('customer_transaction_data.csv')
campaign=pd.read_csv('campaign_data.csv')
test=pd.read_csv('test_QyjYwdj.csv')
train=pd.read_csv('train.csv')
customerdemo=pd.read_csv('customer_demographics.csv')
numberchildren_familysize=pd.read_csv('numberchildren_familysize.csv')
marital_status_familysize=pd.read_csv('marital_status_familysize.csv')

# Treating null value in customer_demographic

In [3]:
# it seems that the customerdemo data set have null values so imputing the null values by using conditional statments.
# As we see there are null values in 'marital_status' and 'no_of_children'.
# if we check the ""marital_status"" depends apon two factors like "family_size","no_of_children" if we groups of marital_status 
# on basis of these two factors we will see the trand so replacing the values with the """"""MODE""""""  HIGHEST number of accurance.
# same as no_of_children is dependent on ""marital_status"" and ""family_size"".

In [4]:
# Creating the dataframe for the trend:

In [5]:
numberchildren_familysize

,family_size,1,2,3+
0,1,No_values,No_values,No_values
1,2,Mode Single,No_values,No_values
2,3,Mode Married,Mode Single,No_values
3,4,No_values,Single,Filled Single
4,5+,No_values,No_values,Mode Married


In [6]:
marital_status_familysize

,family_size,Married,single
0,1,No_values,Mode 0
1,2,Mode 0,Mode 1
2,3,Already Filled 1,Already Filled 1 and 2
3,4,Filled by 2,Already Filled by 2 and 3
4,5+,3+ Already Filled,3+ Already Filled


In [7]:
customerdemo1=customerdemo.copy()

In [8]:
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='3') & (customerdemo1['no_of_children']=='2'),'Single',customerdemo1['marital_status'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='2') & (customerdemo1['no_of_children']=='1'),'Single',customerdemo1['marital_status'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='4') & (customerdemo1['no_of_children']=='2'),'Married',customerdemo1['marital_status'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='3') & (customerdemo1['no_of_children']=='1'),'Married',customerdemo1['marital_status'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='5+') & (customerdemo1['no_of_children']=='3+'),'Married',customerdemo1['marital_status'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['family_size']=='4') & (customerdemo1['no_of_children']=='3+'),'Single',customerdemo1['marital_status'])
customerdemo1['no_of_children']=np.where((customerdemo1['no_of_children'].isnull()) & (customerdemo['family_size']=='1') & (customerdemo['marital_status']=='Single'),'0',customerdemo1['no_of_children'])
customerdemo1['no_of_children']=np.where((customerdemo1['no_of_children'].isnull()) & (customerdemo['family_size']=='2') & (customerdemo['marital_status']=='Married'),'0',customerdemo1['no_of_children'])
customerdemo1['no_of_children']=np.where((customerdemo1['no_of_children'].isnull()) & (customerdemo['family_size']=='2') & (customerdemo['marital_status']=='Single'),'1',customerdemo1['no_of_children'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['no_of_children'].isnull()) & (customerdemo1['family_size']=='1'),'Single',customerdemo1['marital_status'])
customerdemo1['no_of_children']=np.where((customerdemo1['marital_status']=='Single') & (customerdemo1['no_of_children'].isnull()) & (customerdemo1['family_size']=='1'),'0',customerdemo1['no_of_children'])
customerdemo1['marital_status']=np.where((customerdemo1['marital_status'].isnull()) & (customerdemo1['no_of_children'].isnull()) & (customerdemo1['family_size']=='2'),'Married',customerdemo1['marital_status'])
customerdemo1['no_of_children']=np.where((customerdemo1['marital_status']=='Married') & (customerdemo1['no_of_children'].isnull()) & (customerdemo1['family_size']=='2'),'0',customerdemo1['no_of_children'])

In [9]:
customerdemo1.isnull().sum()

customer_id       0
age_range         0
marital_status    0
rented            0
family_size       0
no_of_children    0
income_bracket    0
dtype: int64

In [10]:
# seems that there are no more null value anymore.

# Treating campaign dataset:

In [11]:
campaign['start_date']=pd.to_datetime(campaign['start_date'])
campaign['end_date']=pd.to_datetime(campaign['end_date'])
days_c=(campaign['end_date']-campaign['start_date']).astype('timedelta64[D]')
campaign['start_date']=campaign['start_date'].astype('object')
campaign['end_date']=campaign['end_date'].astype('object')
for i in range(len(days_c)):
    if days_c[i]<0:
        a=campaign['start_date'][i]
        campaign['start_date'].replace(campaign['start_date'][i],campaign['end_date'][i],inplace=True)
        campaign['end_date'].replace(campaign['end_date'][i],a,inplace=True)
campaign['start_date']=pd.to_datetime(campaign['start_date'])
campaign['end_date']=pd.to_datetime(campaign['end_date'])
campaign['days']=(campaign['end_date']-campaign['start_date']).astype('timedelta64[D]')

In [12]:
# In campaign dataset we have some date wrongly misplaced as in place of start date there is end date and visa varsa.
# replacing them by calculating the diffrance between the dates if the diffrance is negative that the there is any error.
##### ONE IMPORTANT FEATURE CAN BE EXTRACTED FROM THIS TABLE THAT IS "NUMBER OF DAYS THE CAMPAIGN WENT ON".

# Treating customer_transcation dataset:

In [13]:
customer['total_paid']=customer['selling_price']-(abs(customer['other_discount']))-(abs(customer['coupon_discount']))

In [14]:
# TOTAL amount spend by the customer can be one of the feature extracted.
# Reaming datasets have no null values and clean so no treatment needed. 

# Merging the datsets

In [15]:
coupon_item=pd.merge(coupon,item_data,how='inner',on='item_id')
data=pd.merge(coupon_item,customer,how='inner',on='item_id')
data.drop(columns=['brand_type','category'],inplace=True)

In [16]:
# Here we are merging the coupon data set with the Item_dataset as """coupon_item""" so method is inner join to get the comman columns.
# second we are merging coupon_item table and customer_transaction_data as """data""".

In [17]:
data.head()

,coupon_id,item_id,brand,date,customer_id,quantity,selling_price,other_discount,coupon_discount,total_paid
0,105,37,56,2012-03-11,1044,1,47.73,-11.04,0.0,36.69
1,105,37,56,2012-05-15,1044,3,142.48,-33.84,0.0,108.64
2,105,37,56,2012-08-23,1044,3,142.48,-27.43,0.0,115.05
3,105,37,56,2012-10-09,1044,2,95.11,-18.17,0.0,76.94
4,105,37,56,2012-10-26,1044,3,169.91,0.00,0.0,169.91


In [18]:
data['other_discount']=data['other_discount'].apply(lambda x : -1*x if x<0 else x)
data['coupon_discount']=data['coupon_discount'].apply(lambda x : -1*x if x<0 else x)

In [19]:
## We are removing the '[-]' symbole from the other_discount,coupon_discount columns.

# Creating new columns feature extraction:

In [20]:
quantity_count=data.pivot_table(values='quantity',index=['customer_id','coupon_id'],aggfunc='sum')
quantity_count.reset_index(inplace=True)
item_count=data.pivot_table(values='item_id',index=['customer_id','coupon_id'],aggfunc='count')
item_count.reset_index(inplace=True)
data.drop(columns=['quantity','item_id','brand'],inplace=True)
final=pd.pivot_table(values=['selling_price','other_discount','coupon_discount','total_paid'],index=['customer_id','coupon_id'],data=data,aggfunc='sum')
final.reset_index(inplace=True)

In [21]:
total_p=pd.merge(final,item_count,how='left',on=['customer_id','coupon_id'])
total=pd.merge(total_p,quantity_count,how='left',on=['customer_id','coupon_id'])
total.rename(columns={'item_id':'item_count'},inplace=True)

In [22]:
## TWO MORE FEATURES ARE EXTRACED FROM THE TABLE """quantity_count""","""item_count""" AS THERE ARE MANY Transaction for each custome.
## Used pivot table to sum up the quantity and count the number of item customer purchased.

# Concating the test and train data set:

In [23]:
full_data=pd.concat([train,test])

In [24]:
full_data.reset_index(inplace=True)

In [25]:
full_data.drop(columns=['index','id'],inplace=True)

# Merging the Test-Train-campaign datsets:

In [26]:
train_camp=pd.merge(full_data,campaign,how='left',on='campaign_id')
real_data=pd.merge(train_camp,total,how='left',on=['customer_id','coupon_id'])
real_data.drop(columns=['start_date','end_date'],inplace=True)
real_data=pd.get_dummies(data=real_data,columns=['campaign_type'])

In [27]:
# Merging the Test-Train-campaign datsets.
# After Merging it seems that there are so many nullvalues so our main objective is to impute the null values.

In [28]:
real_data['redemption_status']=np.where(real_data['redemption_status'].isnull(),100000,real_data['redemption_status'])
real_data2=real_data.copy()

In [29]:
# replacing the null values in the redemption_status as 100000.

# Filling the null values with zero:

In [30]:
real_data=real_data.fillna(0)

In [31]:
# real_data we have so many null values in coupon_discount,other_discount,selling_price,total_paid so replacing the values with "0" 
# because if the customer did not bought anu thing and the redemption status is "0" that mean no transction is done by the customer 
# so we can replace values with zero but train data do not have redemption Status so there is SECOND METHOD TOO WHICH IS MENTIONED BELOW.

In [32]:
alldata=pd.merge(real_data,customerdemo1,how='left',on='customer_id')

In [33]:
alldata.isnull().sum()

campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
days                     0
coupon_discount          0
other_discount           0
selling_price            0
total_paid               0
item_count               0
quantity                 0
campaign_type_X          0
campaign_type_Y          0
age_range            53995
marital_status       53995
rented               53995
family_size          53995
no_of_children       53995
income_bracket       53995
dtype: int64

# Claning the extra anomalies inside the columns:

In [34]:
alldata['family_size']=alldata['family_size'].apply(lambda x: str(x).replace('+','') if '+' in str(x) else str(x))
alldata['no_of_children']=alldata['no_of_children'].apply(lambda x: str(x).replace('+','') if '+' in str(x) else str(x))
alldata['family_size']=alldata['family_size'].astype('float')
alldata['no_of_children']=alldata['no_of_children'].astype('float')

In [35]:
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('18-25','1') if str(x)=='18-25' else str(x))
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('26-35','2') if str(x)=='26-35' else str(x))
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('36-45','3') if str(x)=='36-45' else str(x))
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('46-55','4') if str(x)=='46-55' else str(x))
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('56-70','5') if str(x)=='56-70' else str(x))
alldata['age_range']=alldata['age_range'].apply(lambda x: str(x).replace('70+','6') if str(x)=='70+' else str(x))
alldata['marital_status']=alldata['marital_status'].apply(lambda x: str(x).replace('Single','1') if str(x)=='Single' else str(x))
alldata['marital_status']=alldata['marital_status'].apply(lambda x: str(x).replace('Married','2') if str(x)=='Married' else str(x))

In [36]:
alldata['age_range']=alldata['age_range'].astype('float')
alldata['marital_status']=alldata['marital_status'].astype('float')

In [37]:
## As there is null values in ""age_range"",""marital_status"" so can not lable encode.
## as there are less number of catagories so did the encoding manually.

# Imputation of the data with the group Median:

In [38]:
data112=alldata[['rented','family_size','no_of_children','income_bracket','age_range', 'marital_status']].fillna(alldata.groupby(['campaign_id','coupon_id'])[['rented','family_size','no_of_children','income_bracket','age_range', 'marital_status']].transform('median'))

In [39]:
data=alldata[['campaign_id', 'coupon_id', 'customer_id', 'redemption_status', 'days',
       'coupon_discount', 'other_discount', 'selling_price', 'total_paid',
       'campaign_type_X', 'campaign_type_Y']]
data[['rented','family_size','no_of_children','income_bracket','age_range','marital_status']]=data112

In [40]:
## replacing the null values in there columns with the """FILL NA BY APPLY AND GROUP BY function""" so with this way the
## first grouping the is done and then the null values are fill with respective group ""Median"".

# Splliting the full_datset into Test Train:

In [41]:
test1=data[data['redemption_status']==100000]
train1=data[data['redemption_status']!=100000]
test1.drop(columns='redemption_status',inplace=True)

# Model Building:

In [42]:
X=train1.drop(columns='redemption_status')
y=train1['redemption_status']

# PCA on test-train:

In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
sc=StandardScaler()
pca=PCA()

scaledx=sc.fit_transform(X)
scaledtest=sc.transform(test1)

pcax=pca.fit_transform(X)
pcatest=pca.transform(test1)

In [44]:
## Did pca because with the help of pair plor able  to detemine the relations between the features  and there is high 
## multicollinearity and pca reduces dimensions and mmulticollinearity.

In [45]:
model=lm.fit(pcax,y)
y_prob=model.predict_proba(pcatest)[:,1]

In [46]:
#Used ""XGBOOST With Default Parameter"" and build the model used randomforest too but on xgboost getting high scores and area under the curve.

In [47]:
# got 89.8 area under the roc cure for public data.
#got 86.08 area under the roc cure for Private data.

# Second way of imputing the values (2):

In [48]:
alldata1=pd.merge(real_data2,customerdemo1,how='left',on='customer_id')

In [49]:
fill_columns=alldata1[['coupon_discount','other_discount','selling_price','total_paid','item_count','quantity']].fillna(alldata1.groupby(['coupon_id','family_size'])[['coupon_discount','other_discount','selling_price','total_paid','item_count','quantity']].transform('median'))

In [50]:
all=alldata1[['campaign_id', 'coupon_id', 'customer_id', 'redemption_status', 'days','campaign_type_X', 'campaign_type_Y',
       'age_range', 'marital_status', 'rented', 'family_size',
       'no_of_children', 'income_bracket']]

In [51]:
all[['coupon_discount','other_discount','selling_price','total_paid','quantity','item_count']]=fill_columns

In [52]:
all['family_size']=all['family_size'].apply(lambda x: str(x).replace('+','') if '+' in str(x) else str(x))
all['no_of_children']=all['no_of_children'].apply(lambda x: str(x).replace('+','') if '+' in str(x) else str(x))
all['family_size']=all['family_size'].astype('float')
all['no_of_children']=all['no_of_children'].astype('float')

In [53]:
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('18-25','1') if str(x)=='18-25' else str(x))
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('26-35','2') if str(x)=='26-35' else str(x))
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('36-45','3') if str(x)=='36-45' else str(x))
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('46-55','4') if str(x)=='46-55' else str(x))
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('56-70','5') if str(x)=='56-70' else str(x))
all['age_range']=all['age_range'].apply(lambda x: str(x).replace('70+','6') if str(x)=='70+' else str(x))
all['marital_status']=all['marital_status'].apply(lambda x: str(x).replace('Single','1') if str(x)=='Single' else str(x))
all['marital_status']=all['marital_status'].apply(lambda x: str(x).replace('Married','2') if str(x)=='Married' else str(x))
all['age_range']=all['age_range'].astype('float')
all['marital_status']=all['marital_status'].astype('float')

In [54]:
data1995=all[['rented','family_size','no_of_children','income_bracket','age_range', 'marital_status','coupon_discount','other_discount','selling_price','total_paid','quantity','item_count']].fillna(all.groupby(['campaign_id','coupon_id'])[['rented','family_size','no_of_children','income_bracket','age_range', 'marital_status','coupon_discount','other_discount','selling_price','total_paid','quantity','item_count']].transform('median'))

In [55]:
dat=all[['campaign_id','coupon_id','customer_id','redemption_status','days','campaign_type_X','campaign_type_Y']]

In [56]:
dat[['rented','family_size','no_of_children','income_bracket','age_range', 'marital_status','coupon_discount','other_discount','selling_price','total_paid','quantity','item_count']]=data1995

In [57]:
dat['coupon_discount'].fillna(dat['coupon_discount'].median(),inplace=True)
dat['other_discount'].fillna(dat['other_discount'].median(),inplace=True)
dat['selling_price'].fillna(dat['selling_price'].median(),inplace=True)
dat['total_paid'].fillna(dat['total_paid'].median(),inplace=True)
dat['quantity'].fillna(dat['quantity'].median(),inplace=True)
dat['item_count'].fillna(dat['item_count'].median(),inplace=True)

# Splliting the full_datset into Test Train:

In [58]:
test2=dat[dat['redemption_status']==100000]
train2=dat[dat['redemption_status']!=100000]
test2.drop(columns='redemption_status',inplace=True)

# Model Building:

In [59]:
X=train2.drop(columns='redemption_status')
y=train2['redemption_status']

In [60]:
model=lm.fit(X,y)
y_prob=model.predict_proba(test2)[:,1]

In [61]:
# got 81.4 area under the roc cure for public data.
#got 81.5 area under the roc cure for Private data.

In [62]:
#---------------------------------------------------------END-------------------------------------------------------------------

In [63]:
#Featuring engneering is not done like:selection of features .
#but did Feature extration as added few new features to the table as it place the vital role in preducting the given target.
#For feature engneering we can use SKLEARN.FEATURE SELECTION.

In [ ]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#bestfeatures = SelectKBest()
#fit = bestfeatures.fit()
#dfscores = pd.DataFrame(fit.scores_)

In [64]:
#Once feature engneering can agin run the model with those columns into consuderation.